In [19]:
import pandas
from pandas import DataFrame

In [43]:
class Get_Vendor_Info():
    def __init__(self,path='C:\\Users\\user\\anaconda3\\Projects\\BizCases\\data/製造商-清單.JSON',load=True):
        self.app_config=None
        if load==True:
            with codecs.open(path,'r',encoding='utf-8') as R: #讀到檔案傳到R
                self.app_config=json.load(R)
    
    #依據網站規範而定(功能物件)
    def get_page(self, page_no=1):
        url='%s/%s?page=%d'%( 'http://www.edomtech.com/tw','/manufacturers/index.php',page_no)
        response = requests.get(url)
        if not response.ok:
            return None
    
        soup = BeautifulSoup(response.text, 'html.parser')
        result = soup.find_all('div', {'class': 'Txt b-box'})
#作業
        company_info = dict()

        for ele in result:
            if ele.text.strip() == '':
                continue

            company_id = ''
            company_name = ''

            for tag in ele.find_all():
                if tag.name == 'a':
                    company_id = tag['title']
                elif tag.name == 'p':
                    if tag.text.strip() != '':
                        company_name = tag.text
                    
            if company_name == '':
                company_info[company_id] = company_id
            else:
                 company_info[company_id] = company_name
        return DataFrame(data={ 'company_id': company_info.keys(), 'company_name': company_info.values() })

    def get_all_pages(self,max_num_page=7):
        vendor_list=None
        for page_no in range(1,max_num_page):
            df=self.get_page(page_no)
            if df is None:
                break
            if vendor_list is None:
                vendor_list=df
            else:
                vendor_list=pandas.concat([vendor_list,df],axis=0)
            
        return vendor_list

In [44]:
worker=Get_Vendor_Info()
worker.get_page(3)

,company_id,company_name
0,Kinetic Technologies,芯凱電子
1,Kingston Technology,金士頓
2,Knowles,樓氏電子有限公司
3,Lattice,美商萊迪思半導體股份有限公司
4,Leahkinn,力勤科技有限公司
5,LG Chem,樂金化學
6,LG Innotek,樂金依諾特
7,Libre Wireless Technologies,Libre Wireless Technologies
8,Littelfuse,利特電子有限公司
9,Longsys,江波龍電子股份有限公司


In [45]:
vendors=worker.get_all_pages()
vendors.head()

,company_id,company_name
0,Advanced Analog Technology,台灣類比科技
1,Airgain,Airgain
2,Akoustis,Akoustis
3,ams OSRAM,艾邁斯歐司朗半導體國際股份有限公司
4,Analogix,矽谷數模半導體公司


In [46]:
vendors.tail()

,company_id,company_name
0,Valencell,Valencell
1,Western Digital (SanDisk),Western Digital (SanDisk)
2,Wise-Integration,法商懷智整合股份有限公司
3,Xilinx,Xilinx
4,xMEMS Labs,xMEMS Labs


In [47]:
vendors.to_csv('供應商資料表.csv',index=False) #作業